In [7]:
import numpy as np
import logging

logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [8]:
_num_tuples = 1296
_num_features = 11
_label_dims = 45

In [31]:
def load_data(files, num_tuples, num_features, label_dims):
    data_file, label_file = files

    data = np.zeros((num_tuples, num_features))
    labels = np.zeros((num_tuples, label_dims))
    
    with open(data_file, 'rb') as f:
        for i,line in enumerate(f.readlines()):
            ##################[ISSUE! @YUZHE]HARD CODED, PLEASE CHECK THE DATA FORMAT ########################
            processed = line.rstrip('\n').rstrip('\r').rstrip(',,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,').split(',')
            ###################################################################################################
            #logger.info(processed)
            try:
                assert num_features == len(processed)
                data[i,:] = processed
            except AssertionError as err:
                logger.info("Wrong Feature Number claimed !, {}, {}".format(num_features, len(processed)))
                
    with open(label_file, 'rb') as f:
        for i,line in enumerate(f.readlines()):
            processed = line.rstrip('\n').split(',')
            try:
                assert label_dims == len(processed)
                labels[i,:] = processed
            except AssertionError as err:
                logger.info("Wrong Label Dimensions claimed !")
        return data, labels

In [32]:
data, labels = load_data(('temperature.csv', 'spectrum.csv'), _num_tuples, _num_features, _label_dims)

In [71]:
class Dataset(object):
    def __init__(self, data):
        # m, n denote number of tuples and features respectively
        self._m = data[0].shape[0]
        self._n = data[0].shape[1]
        self._training_data = data[0]
        self._training_labels = data[1]
        
    def __len__(self):
        return self._m
    
    def __getitem__(self, idx):
        return  self._training_data[idx,:], self._training_labels[idx,:] 

    def fetch_col(self, col_index):
        return self._training_data[:, col_index]

    def shuffle(self, seed=None):
        if seed:
            np.random.seed(seed=seed)
        shuffled_indices = np.arange(self._m)
        np.random.shuffle(shuffled_indices)
        self._training_data = np.take(self._training_data, shuffled_indices, axis=0)
        self._training_labels = np.take(self._training_labels, shuffled_indices)

    @property
    def num_tuples(self):
        return self._m

    @property
    def num_features(self):
        return self._n

    @property
    def labels(self):
        return self._training_labels

    @property
    def data_table(self):
        return self._training_data

In [72]:
dataset = Dataset((data, labels))

In [77]:
import torch
from torch import nn

# model definition
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(11, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 45)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x
    def name(self):
        return 'nn'

In [84]:
from torch.utils.data import DataLoader
import torch.nn.functional as F

args = {'lr':0.001, 
        'max_steps':1000,
        'batch_size':128,
        'epoch':10,
        'enable_gpu':None}
train_loader = DataLoader(dataset, batch_size=args['batch_size'], shuffle=True)

In [85]:
import torch.optim as optim

device = torch.device("cpu")
model = NN().to(device)
optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=0.9)

In [94]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.float().to(device), target.float().to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.mse_loss(output, target)
        loss.backward()
        optimizer.step()
        logger.info('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.mse_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    logger.info('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [96]:
for epoch in range(1, args['epoch'] + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    #test(args, model, device, test_loader)

INFO:root:Train Epoch: 1 [0/1296 (0%)]	Loss: 0.431346
INFO:root:Train Epoch: 1 [128/1296 (9%)]	Loss: 0.422769
INFO:root:Train Epoch: 1 [256/1296 (18%)]	Loss: 0.420668
INFO:root:Train Epoch: 1 [384/1296 (27%)]	Loss: 0.402756
INFO:root:Train Epoch: 1 [512/1296 (36%)]	Loss: 0.400048
INFO:root:Train Epoch: 1 [640/1296 (45%)]	Loss: 0.399653
INFO:root:Train Epoch: 1 [768/1296 (55%)]	Loss: 0.396001
INFO:root:Train Epoch: 1 [896/1296 (64%)]	Loss: 0.377876
INFO:root:Train Epoch: 1 [1024/1296 (73%)]	Loss: 0.378002
INFO:root:Train Epoch: 1 [1152/1296 (82%)]	Loss: 0.379172
INFO:root:Train Epoch: 1 [160/1296 (91%)]	Loss: 0.377841
INFO:root:Train Epoch: 2 [0/1296 (0%)]	Loss: 0.377784
INFO:root:Train Epoch: 2 [128/1296 (9%)]	Loss: 0.377779
INFO:root:Train Epoch: 2 [256/1296 (18%)]	Loss: 0.377782
INFO:root:Train Epoch: 2 [384/1296 (27%)]	Loss: 0.377827
INFO:root:Train Epoch: 2 [512/1296 (36%)]	Loss: 0.377921
INFO:root:Train Epoch: 2 [640/1296 (45%)]	Loss: 0.378026
INFO:root:Train Epoch: 2 [768/1296 (5